In [71]:
import os
from dotenv import load_dotenv
from google.analytics.data_v1beta import BetaAnalyticsDataClient 
from google.analytics.data_v1beta.types import DateRange, Metric, Dimension, RunReportRequest
from google.oauth2 import service_account


# LOADING PROPERTY_ID AND GA4 CREDENTIALS JSON FILE
#load_dotenv(r"C:\Users\garvi\Documents\Data Science Projects\Digital Marketing Dashboard\.env")
PROPERTY_ID = os.getenv("Property_Id")
CREDENTIALS_GA4_JSON = os.getenv("GA4_Service_Account_Credentials", r"C:\Users\garvi\Documents\Data Science Projects\Digital Marketing Dashboard\eastern-perigee-469018-v9-ac5502363b52.json" )

print(os.getenv("GA4_Service_Account_Credentials"))
print(os.getenv("Property_Id"))

# intializing client , request and responmse to fetch data from google analytics 4 with property Id and service credentials json

credentials = service_account.Credentials.from_service_account_file(CREDENTIALS_GA4_JSON)
client = BetaAnalyticsDataClient(credentials = credentials)

request = RunReportRequest(
    property  = f"properties/{PROPERTY_ID}",
    dimensions = [Dimension(name="city")],
    metrics = [Metric(name="activeUsers")],
    date_ranges = [DateRange(start_date = "2025-08-15", end_date = "2025-08-15")]
)

response = client.run_report(request)
#print(response)

for row in response.rows:
    print({dim.name: value.value for dim, value in zip(request.dimensions, row.dimension_values)},
          {met.name: value.value for met, value in zip(request.metrics, row.metric_values)})
    
 



C:\Users\garvi\Documents\Data Science Projects\Digital Marketing Dashboard\eastern-perigee-469018-v9-ac5502363b52.json
501340180
{'city': 'Hansi'} {'activeUsers': '5'}
